In [1]:
import torch
from ultralytics import YOLO

In [2]:
class YoloClient:
    def __init__(self, model: YOLO, dataset: str):
        self.model = model
        self.config_path = f"datasets/{dataset}/config.yaml"

    def train(self, epochs):
        self.model.train(data=self.config_path, epochs=epochs)
    
    def set_parameters(self, state_dict):
        self.model.load_state_dict(state_dict)

    def get_parameters(self):
        return self.model.state_dict()

In [3]:
class YoloServer:
    def __init__(self, model: YOLO, datasets):
        self.model = model
        self.clients = [YoloClient(self.model, dataset) for dataset in datasets]

    def start(self, rounds=1, epochs=1):
        for round in range(rounds):
            print(f"Starting round {round+1}/{rounds}")
            for client in self.clients:
                client.set_parameters(self.model.state_dict())
                client.train(epochs)
            self.update([client.get_parameters() for client in self.clients])
    
    def update(self, state_dicts):
        print("Updating global model")
        avg_state_dict = {key: torch.zeros_like(value) for key, value in self.model.state_dict().items()}
        
        for state_dict in state_dicts:
            for key in avg_state_dict.keys():
                avg_state_dict[key] += state_dict[key]
        
        for key in avg_state_dict.keys():
            avg_state_dict[key] /= len(state_dicts)
        
        self.model.load_state_dict(avg_state_dict)


In [4]:
model = YOLO("runs/detect/train3/weights/best.pt")
datasets = ["citypersons", "deepdrive", "kitti", "roadsigns"]
server = YoloServer(model, datasets)
server.start()

Starting round 1/1
New https://pypi.org/project/ultralytics/8.2.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.33 🚀 Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train3/weights/best.pt, data=datasets/citypersons/config.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

/home/pierre/venvs/base/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /home/pierre/git/edgeAI/datasets/citypersons/labels/train... 2975 images, 488 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:07<00:00, 378.09it/s]

train: New cache created: /home/pierre/git/edgeAI/datasets/citypersons/labels/train.cache


...

train: New cache created: /home/pierre/git/edgeAI/datasets/roadsigns/labels/train.cache



val: Scanning /home/pierre/git/edgeAI/datasets/roadsigns/labels/val... 176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:01<00:00, 125.17it/s]


val: New cache created: /home/pierre/git/edgeAI/datasets/roadsigns/labels/val.cache
Plotting labels to runs/detect/train25222/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000476, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train25222
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.54G      1.054      2.451      1.138         26        640: 100%|██████████| 44/44 [00:06<00:00,  6.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/6 [00:00<?, ?it/s]/home/pierre/venvs/base/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 2/6 [00:00<00:01,  2.58it/s]

: 